# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846782699239                   -0.70    4.8         
  2   -7.852311006509       -2.26       -1.53    1.0   28.7ms
  3   -7.852614718409       -3.52       -2.56    1.5   31.9ms
  4   -7.852645894147       -4.51       -2.87    2.5   39.7ms
  5   -7.852646461311       -6.25       -3.13    1.0   31.0ms
  6   -7.852646677637       -6.66       -4.07    1.0   30.8ms
  7   -7.852646686680       -8.04       -5.08    2.2   41.2ms
  8   -7.852646686724      -10.35       -5.38    1.5   35.3ms
  9   -7.852646686728      -11.36       -5.60    1.2   33.1ms
 10   -7.852646686730      -11.77       -6.57    1.0   31.8ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846791647342                   -0.70           4.5         
  2   -7.852526678919       -2.24       -1.64   0.80    2.2    403ms
  3   -7.852635670256       -3.96       -2.73   0.80    1.0   28.1ms
  4   -7.852646435994       -4.97       -3.24   0.80    2.2   37.9ms
  5   -7.852646663994       -6.64       -4.05   0.80    1.2   29.5ms
  6   -7.852646686347       -7.65       -4.69   0.80    2.0   35.6ms
  7   -7.852646686718       -9.43       -5.44   0.80    1.8   34.7ms
  8   -7.852646686729      -10.93       -6.86   0.80    2.0   35.2ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.449062e+01     3.459514e+00
 * time: 0.5785741806030273
     1     1.296408e+00     1.719668e+00
 * time: 0.8134510517120361
     2    -1.782898e+00     2.045837e+00
 * time: 0.8426899909973145
     3    -3.797214e+00     1.743141e+00
 * time: 0.8849210739135742
     4    -5.285085e+00     1.605613e+00
 * time: 0.9270901679992676
     5    -6.813404e+00     7.702941e-01
 * time: 0.9693081378936768
     6    -6.975342e+00     9.654320e-01
 * time: 0.9986531734466553
     7    -7.484278e+00     6.218531e-01
 * time: 1.0278480052947998
     8    -7.674467e+00     3.284689e-01
 * time: 1.0570399761199951
     9    -7.789492e+00     1.257377e-01
 * time: 1.0864901542663574
    10    -7.822904e+00     1.035072e-01
 * time: 1.1161420345306396
    11    -7.838553e+00     8.606433e-02
 * time: 1.1455771923065186
    12    -7.846841e+00     5.507149e-02
 * time: 1.1747090816497803
    13    -7.847983e+00     4.215131e-02
 * time: 1.2039780616

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846605959555                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645897712                   -1.64         
  2   -7.852646686730       -6.10       -3.71    1.91s
  3   -7.852646686730      -13.30       -7.24    204ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.9918588011265883e-7
|ρ_newton - ρ_scfv| = 3.387686814355384e-7
|ρ_newton - ρ_dm|   = 7.57048989439487e-10
